# `SimPEG(emg3d)`

Requires
- `simpeg>=0.22.1`
- `emg3d@inv-simpeg` (`pip install git+https://github.com/emsig/emg3d@inv-simpeg`

Very simple plus all possible inv-crimes (same mesh, some solver, ...).

For the "original" versions, see:
- https://curvenote.com/@prisae/emg3d-as-solver-for-simpeg/first-try [the very original]
- https://curvenote.com/@prisae/emg3d-as-solver-for-simpeg/simpeg-emg3d-inversion-automatic-grid [original with automatic gridding]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import emg3d
import simpeg
import emg3d.inversion.simpeg as isimpeg

%run ../data/sampledata

In [ ]:
%matplotlib widget

## Load Simulation (incl. Survey, Models, Data)

In [ ]:
sim, model = load_data("../data/SimPEG-Original")
model_init = sim.model.copy()

# sim.tol_gradient = 1e-2  # could be reduced further to potentially speed-up more

## Define active cells, instantiate FDEMSimulation and Inversion

In [ ]:
# Create active cells
inds_active = np.zeros(sim.model.shape, dtype=bool)
inds_active[:, :, :-6] = True

# Instantiate SimPEG Simulation
FDEMSimulation = isimpeg.FDEMSimulation(simulation=sim, active_indices=inds_active)

# Define directives
starting_beta = simpeg.directives.BetaEstimate_ByEig(beta0_ratio=1)
beta_schedule = simpeg.directives.BetaSchedule(coolingFactor=2, coolingRate=1)
target_misfit = simpeg.directives.TargetMisfit(chifact=1)

# Instantiate Inversion
INV = isimpeg.Inversion(
    simulation=FDEMSimulation,
    maxIter=30,
    optimization_opts={
        'maxIterLS': 20,
        'maxIterCG': 20,
        'tolCG': 1e-3,
    },
    regularization_opts={
        'alpha_s': 1e-8,
        'alpha_x': 1,
        'alpha_y': 10,
        'alpha_z': 1,
    },
    directiveList=[
        starting_beta,
        beta_schedule,
        target_misfit,
    ],
)

## Run inversion

In [ ]:
INV.run()

## Plots

In [ ]:
plot_models(sim, model_init, model, zind=6, vmin=0.33, vmax=100)

In [ ]:
plot_responses(sim)

In [ ]:
emg3d.Report()